In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/3.part'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

directory_in = './file/2.part_number'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part={crawler_series['Part Number']}&func=PART&vid=''', crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_dict = resp.json()

                        if dict_dict:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            dict_info = {key.strip().title(): value.strip() if type(value) == str else value for key, value in dict_dict['pd'].items() if key not in ['rowId', 'site', 'webBase', 'basePart', 'maxRows'] and value != None}

            # = = =  = = =  = = =

            vehicle = '\n'.join([dict_['buyersGuideDesc'].strip() for dict_ in dict_dict['pp']['buyersGuides']])

            # = = =  = = =  = = =

            dict_key, list_dict = {}, dict_dict['pp']['partSpecs']
            for dict_ in list_dict:
                key = dict_['attributeName_en'].strip()
                if key in dict_key:
                    dict_key[key] += 1
                else:
                    dict_key[key] = 1

            dict_specification, dict_count = {}, {}
            for i, dict_ in enumerate(list_dict):
                key = dict_['attributeName_en'].strip()
                if dict_key[key] == 1:
                    dict_specification[str(i)] = {key: dict_['siteValue_en'].strip()}
                else:
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                    dict_specification[str(i)] = {f'{key}-{dict_count[key]}': dict_['siteValue_en'].strip()}

            # = = =  = = =  = = =

            dict_type, list_dict = {}, dict_dict['pp']['categories']
            for i, dict_ in enumerate(list_dict):
                dict_type[str(i)] = dict_['categoryName_en'].strip()

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Part Number': crawler_series['Part Number'],
                                          'Vehicle': vehicle,
                                          'Picture': '',
                                          'Url': crawler_series['Url'],
                                          'Json_Type': json.dumps(dict_type),
                                          'Json_Info': json.dumps(dict_info),
                                          'Json_Specification': json.dumps(dict_specification)})

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求1次] - [剩余29条] - [1（100.00%）] - [2024/12/11 21:53:45] - 392. https://www.standardbrand.com/en/ecatalog?part=APS627&type=p&search=s

[True] - [请求1次] - [剩余28条] - [1（100.00%）] - [2024/12/11 21:53:46] - 440. https://www.standardbrand.com/en/ecatalog?part=APS708&type=p&search=s

[True] - [请求1次] - [剩余27条] - [1（100.00%）] - [2024/12/11 21:53:46] - 351. https://www.standardbrand.com/en/ecatalog?part=APS584&type=p&search=s

[True] - [请求1次] - [剩余26条] - [1（100.00%）] - [2024/12/11 21:53:46] - 439. https://www.standardbrand.com/en/ecatalog?part=APS707&type=p&search=s

[True] - [请求1次] - [剩余25条] - [1（100.00%）] - [2024/12/11 21:53:46] - 421. https://www.standardbrand.com/en/ecatalog?part=APS666&type=p&search=s

[True] - [请求1次] - [剩余24条] - [1（100.00%）] - [2024/12/11 21:53:46] - 449. https://www.standardbrand.com/en/ecatalog?part=APK102&type=p&search=s

[True] - [请求1次] - [剩余23条] - [1（100.00%）] - [2024/12/11 21:53:47] - 435. https://www.standardbrand.com/en/ecatalog?part=APS703&type=p&search=s
